### Import thư viện

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier

from xgboost import XGBClassifier
import cupy as cp

### Kiểm tra dữ liệu

In [3]:
data = pd.read_csv('../dataset/data.csv')
print(f"Kích thước dữ liệu: {data.shape}")
print(f"Số cột dữ liệu: {data.shape[1]}")
print(f"Số dòng dữ liệu: {data.shape[0]}")

print("\nMẫu dữ liệu:")
data.head()

Kích thước dữ liệu: (21955, 168)
Số cột dữ liệu: 168
Số dòng dữ liệu: 21955

Mẫu dữ liệu:


,diseases,shortness of breath,sharp chest pain,dizziness,insomnia,chest tightness,palpitations,irregular heartbeat,breathing fast,hoarse voice,...,back stiffness or tightness,skin pain,low back stiffness or tightness,nose deformity,sore in nose,hip weakness,back swelling,ankle stiffness or tightness,ankle weakness,neck weakness
0,vocal cord polyp,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,vocal cord polyp,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,vocal cord polyp,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,vocal cord polyp,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,vocal cord polyp,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Kiểm tra giá trị null

In [4]:
null_counts = data.isnull().sum()
print(f"Tổng số giá trị null: {null_counts.sum()}")

Tổng số giá trị null: 0


#### Số lượng bệnh và triệu chứng có trong mẫu

In [5]:
disease_counts = data['diseases'].value_counts()

symptom_cols = data.columns[1:] 
symptom_counts = data[symptom_cols].sum().sort_values(ascending=False)

print("Số lượng bệnh: ", disease_counts.shape[0])
print("Số triệu chứng: ", symptom_counts.shape[0])

Số lượng bệnh:  325
Số triệu chứng:  167


#### Danh sách bệnh

In [6]:
disease_list = disease_counts.index.tolist()
with open("../data_info/disease_list.txt", "w", encoding="utf-8") as f:
    for disease in disease_list:
        f.write(f"{disease}\n")
disease_list

['diverticulitis',
 'infectious gastroenteritis',
 'acute pancreatitis',
 'appendicitis',
 'esophagitis',
 'nose disorder',
 'pyelonephritis',
 'cholecystitis',
 'strep throat',
 'acute sinusitis',
 'gum disease',
 'gastritis',
 'vulvodynia',
 'hiatal hernia',
 'sepsis',
 'pleural effusion',
 'acute bronchospasm',
 'flu',
 'noninfectious gastroenteritis',
 'pneumonia',
 'asthma',
 'dental caries',
 'urinary tract infection',
 'acute bronchitis',
 'fibromyalgia',
 'rectal disorder',
 'concussion',
 'chronic back pain',
 'herpangina',
 'choledocholithiasis',
 'seasonal allergies (hay fever)',
 'diverticulosis',
 'pulmonary embolism',
 'sickle cell crisis',
 'acute bronchiolitis',
 'heart attack',
 'gastroesophageal reflux disease (gerd)',
 'gastrointestinal hemorrhage',
 'irritable bowel syndrome',
 'common cold',
 'oral mucosal lesion',
 'viral exanthem',
 'impetigo',
 'mitral valve disease',
 'ischemic heart disease',
 'indigestion',
 'hypokalemia',
 'meningitis',
 'chronic sinusitis',

#### Danh sách triệu chứng

In [7]:
symptom_list = symptom_cols.tolist()
with open("../data_info/symptom_list.txt", "w", encoding="utf-8") as f:
    for symptom in symptom_list:
        f.write(f"{symptom}\n")
symptom_list

['shortness of breath',
 'sharp chest pain',
 'dizziness',
 'insomnia',
 'chest tightness',
 'palpitations',
 'irregular heartbeat',
 'breathing fast',
 'hoarse voice',
 'sore throat',
 'cough',
 'nasal congestion',
 'throat swelling',
 'skin swelling',
 'hip pain',
 'suprapubic pain',
 'blood in stool',
 'back weakness',
 'pus in sputum',
 'flatulence',
 'pus draining from ear',
 'sharp abdominal pain',
 'vomiting',
 'headache',
 'nausea',
 'diarrhea',
 'painful urination',
 'involuntary urination',
 'lower abdominal pain',
 'blood in urine',
 'arm stiffness or tightness',
 'wrist stiffness or tightness',
 'lip swelling',
 'toothache',
 'dry lips',
 'facial pain',
 'mouth ulcer',
 'pain in eye',
 'abnormal movement of eyelid',
 'back pain',
 'neck pain',
 'low back pain',
 'vomiting blood',
 'regurgitation',
 'burning abdominal pain',
 'jaw swelling',
 'mouth dryness',
 'neck swelling',
 'bones are painful',
 'knee weakness',
 'knee stiffness or tightness',
 'muscle pain',
 'difficult

### Huấn luyện mô hình

#### Xử lý dữ liệu và chia tập train / test

In [8]:
import csv

file_path = "../dataset/data_old.csv"
rows_seen = set()
duplicates = []

with open(file_path, "r", encoding="utf-8") as f:
    reader = csv.reader(f)
    header = next(reader)  # Bỏ qua dòng tiêu đề
    for row in reader:
        if len(row) < 2:
            continue  # Bỏ qua dòng trống hoặc không đủ cột
        row_key = tuple(row[1:])  # Bỏ cột đầu tiên khi so sánh
        if row_key in rows_seen:
            duplicates.append(row)
        else:
            rows_seen.add(row_key)

if duplicates:
    print(f"⚠️ Có {len(duplicates)} dòng bị trùng (bỏ qua cột đầu):")

else:
    print("✅ Không có dòng nào bị trùng khi bỏ qua cột đầu.")


✅ Không có dòng nào bị trùng khi bỏ qua cột đầu.


In [53]:
import csv

file_path = "../dataset/data_old.csv"  # File gốc sẽ được ghi đè

rows_seen = set()
unique_rows = []
duplicates = []

with open(file_path, "r", encoding="utf-8") as f:
    reader = csv.reader(f)
    header = next(reader)
    unique_rows.append(header)
    for row in reader:
        if len(row) < 2:
            continue  # Bỏ qua dòng trống hoặc không đủ cột
        row_key = tuple(row[1:])  # Bỏ cột đầu tiên khi so sánh
        if row_key in rows_seen:
            duplicates.append(row)
        else:
            rows_seen.add(row_key)
            unique_rows.append(row)

# Ghi đè lên file cũ
with open(file_path, "w", encoding="utf-8", newline='') as f:
    writer = csv.writer(f)
    writer.writerows(unique_rows)

print(f"✅ Đã loại bỏ {len(duplicates)} dòng trùng lặp. File đã được ghi đè: {file_path}")

✅ Đã loại bỏ 19731 dòng trùng lặp. File đã được ghi đè: ../dataset/data_old.csv


In [10]:
import pandas as pd

# Giả sử y là cột class trong dataframe
df = pd.read_csv("../dataset/data.csv")

# Đếm số lượng mẫu theo class
class_counts = df['diseases'].value_counts()

# Giữ lại các class có ít nhất 2 mẫu
valid_classes = class_counts[class_counts >= 2].index
data = df[df['diseases'].isin(valid_classes)]


In [11]:
X = data.iloc[:, 1:]  # Tất cả các cột trừ cột đầu tiên (diseases)
y = data.iloc[:, 0]   # Cột đầu tiên (diseases)

# Chuyển đổi nhãn bệnh thành số
le = LabelEncoder()
y_encoded = le.fit_transform(y)

print(f"Số lượng nhãn bệnh: {len(le.classes_)}")

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"Kích thước tập huấn luyện: {X_train.shape}")
print(f"Kích thước tập kiểm tra: {X_test.shape}")


Số lượng nhãn bệnh: 289
Kích thước tập huấn luyện: (17535, 167)
Kích thước tập kiểm tra: (4384, 167)


#### Huấn luyện bằng nhiều mô hình

In [12]:
models = {
    'Random Forest': RandomForestClassifier(n_estimators=80, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
}

# Huấn luyện và đánh giá các mô hình
results = {}

for name, model in models.items():
    print(f"\nĐang huấn luyện mô hình {name}...")
    model.fit(X_train, y_train)
    
    # Dự đoán trên tập kiểm tra
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"Độ chính xác của {name}: {accuracy:.4f}")
    
    # Lưu kết quả
    results[name] = {
        'model': model,
        'accuracy': accuracy,
        'y_pred': y_pred
    }


Đang huấn luyện mô hình Random Forest...
Độ chính xác của Random Forest: 0.9190

Đang huấn luyện mô hình Decision Tree...
Độ chính xác của Decision Tree: 0.8739

Đang huấn luyện mô hình Logistic Regression...
Độ chính xác của Logistic Regression: 0.9163


#### So sánh và chọn mô hình tốt nhất

In [13]:
# Tìm mô hình tốt nhất
best_model_name = max(results, key=lambda x: results[x]['accuracy'])
best_model = results[best_model_name]['model']
best_accuracy = results[best_model_name]['accuracy']
best_y_pred = results[best_model_name]['y_pred']

print(f"Mô hình tốt nhất: {best_model_name} với độ chính xác {best_accuracy:.4f}")

# Ma trận nhầm lẫn
print("\nMa trận nhầm lẫn:")
cm = confusion_matrix(y_test, best_y_pred)
print(cm)
    
# Báo cáo chi tiết với precision, recall, F1-score
print("\nBáo cáo chi tiết (Precision, Recall, F1-score):")
report = classification_report(y_test, best_y_pred, zero_division=1)
print(report)


Mô hình tốt nhất: Random Forest với độ chính xác 0.9190

Ma trận nhầm lẫn:
[[ 2  0  0 ...  0  0  0]
 [ 0 10  0 ...  0  0  0]
 [ 0  0  3 ...  0  0  0]
 ...
 [ 0  0  0 ... 76  0  0]
 [ 0  0  0 ...  0 25  0]
 [ 0  0  0 ...  0  0  0]]

Báo cáo chi tiết (Precision, Recall, F1-score):
              precision    recall  f1-score   support

           0       1.00      0.67      0.80         3
           1       0.91      0.91      0.91        11
           2       1.00      1.00      1.00         3
           3       1.00      0.00      0.00         1
           4       0.87      1.00      0.93        13
           5       0.50      0.33      0.40         3
           6       0.97      0.97      0.97        36
           7       1.00      0.96      0.98        45
           8       0.90      0.96      0.93        57
           9       1.00      0.67      0.80         3
          10       1.00      1.00      1.00         3
          11       0.90      0.98      0.94       141
          12     

#### Lưu mô hình

In [14]:
# Tạo thư mục models nếu chưa tồn tại
os.makedirs('../models', exist_ok=True)

# Tối ưu lưu mô hình với nén
model_filename = f'../models/{best_model_name.replace(' ', '_').lower()}_model.joblib'
joblib.dump(best_model, model_filename, compress = 3) 

# Lưu label encoder cũng với nén
encoder_filename = '../models/label_encoder.joblib'
joblib.dump(le, encoder_filename, compress = 3)

print(f"Đã lưu mô hình tại: {model_filename}")
print(f"Đã lưu label encoder tại: {encoder_filename}")

Đã lưu mô hình tại: ../models/random_forest_model.joblib
Đã lưu label encoder tại: ../models/label_encoder.joblib
